Chapter 2.a.iii.  ExprRange reductions
=======

As we saw in the previous chapter, an **ExprRange** adds versatility to **Prove-It** by allowing us to define **Lambda** expressions with an indeterminant number of parameters.  Generally, **ExprRange** expressions allow us to form expressions over an indeterminant number of elements.  Basically, an **ExprRange** is how we formalize the concept of using an ellipsis in a mathematical expression.  For example, $a \cdot (b_1 + \ldots + b_n) = a \cdot b_1 + \ldots + a \cdot b_n$ (distribution) may be expressed using an **ExprRange** on each side of the equation.  Internally, the **ExprRange** has a *parameter* variable for parameterizing each element of the range.  For the LaTeX display, this *parameter* is hidden by default, as in the above example, and the precise parameterization is implicit.  To avoid ambiguity, it can be important at times to display the explicit parameterization.  For the above example, this would be $a \cdot (b_1 + ..b_k.. + b_n) = a \cdot b_1 + ..a \cdot b_k.. + a \cdot b_n$, assuming $k$ is the *parameter* in both **ExprRange** instances.

There are two kinds of reduction rules that are specific to **ExprRange** expressions that are applied in the process of performing a lambda application (beta reduction).
1. *Indexed variable expansions* occur when the **ExprRange** contains one or more **IndexedVar** expressions with an index that is parameterized over the range, and the range of indexed variables is being replaced by an **ExprTuple**.  This falls under two cases.
  1. *Parameter dependent expansion* is required whenever the parameter of the **ExprRange** occurs anywhere besides as an index of an indexed variable.  In that case, the indices of the expanded indexed variable must match the original indices.  For example, $1 \cdot x_1 + \ldots + n \cdot x_n$ could be expanded, under $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, a_{j+1}, \ldots, a_n)$ to $1 \cdot a_1 + \ldots + j \cdot a_j + (j+1) \cdot a_{j+1} + \ldots + n \cdot a_n$ assuming $0 \leq j \leq n$.  However, an error would occur given $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, b_1, \ldots, b_k)$ as the expansion.
  2. *Parameter independent expansion* is allowed otherwise.  In that case, only the lengths of the expansion must match, not the indices themselves.  For example, $x_1 \cdot y_1 + \ldots + x_n \cdot y_n$ could be expanded, under $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, b_1, \ldots, b_k)$, $(y_1, \ldots, y_n) : (c_1, \ldots, c_j, d_1, \ldots, d_k)$ to $a_1 \cdot c_1 + \ldots + a_j \cdot c_j + b_1 \cdot d_1 + \ldots + b_k \cdot d_k$ assuming $j + k = n$.
2. *Range reductions* occur when the **ExprRange** is known to be empty or contain only a single element. 
  1. *Empty range reduction* occurs when the **ExprRange** is known to be empty and may be replaced with zero elements in a containing **ExprTuple**.
  2. *Singular element range reduction* occurs when the **ExprRange** is known to contain only a single element and may be replaced with this one element.

In [1]:
import proveit
%begin exprrange_reductions

## Creating ExprRange expressions

An `ExprRange` may be instantiated with a `parameter`, `body`, `start index`, and `end index`:

In [2]:
from proveit import ExprTuple, ExprRange, defaults
from proveit.numbers import Add, Mult, zero, one, two
from proveit import a, b, c, d, i, j, k, m, n, x, y
a_prod_range = ExprRange(k, Mult(a, k), i, j)

a_prod_range: (a * i), ..(a * k).., (a * j)

In [3]:
a_prod_range.parameter

k

In [4]:
a_prod_range.body

a * k

In [5]:
a_prod_range.start_index

i

In [6]:
a_prod_range.end_index

j

It's sub-expressions are a **Lambda** expression created from the `parameter` and `body`, the `start_index`, and the `end_index`:

In [7]:
a_prod_range.expr_info()

,core type,sub-expressions,expression
0,ExprRange,lambda_map: 1start_index: 2end_index: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJMAAAAUBAMAAAB7fhJGAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAdVJREFU OMuVVDFLw0AUfjVpTGrSdisoQn+BZNNuLoKbDm5W6Q8QG0SR4tIu6iBScHBwsKuL1kkclDhIhQ7J qIvURXSQFoUO0sF3L02jySXFQC6579333b3v7h1A0gb/sw6RzwcHI0rTj05ArBWlpNb9iJR3KOf+ wCSotSipRGAiLU0UrcoZPR0l9cRFkSLzkrmPkjrkokh54+YwG66ULEMIpcSiR8f5gaklA0CohkuN sth7bsscII0dIMoNfuZaiY6LC+IiGhjh+xgeHnEVLjykGreJsosLWQGFNphN9KqgjFQOl5ILaIwN J5QC7V8rVSDKEoDSA9lggW8ml8LfkU64VBx9PDVhj/1bNdJbMImCUvE6WFkWqbCmaA6XWgb1k1aY JmgGiIIJWjoUvaGX+IrRCapfIP2e7MyhoO1WAcYl93ipXWGo7WoXYjWvULFLlBwGDa0iuLjYwWpV 9L4RZB81gx47KFfwmNa9EuwBUeZxRGbjZdMrQdzZBLNOPnDtq7i27JtO8nfbUxnvFCvMDqRwC+eB 2rUALpm8wpGrDiWinPUALnDKecRsZ/uUW86NRK6rRiDQIN//5tHWr11Kk7NJNFgLBmzO1Rcr2S6F cyE///9CRsoPFTB50q+h5OUAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Lambda,parameter: 8body: 4,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAF4AAAAUCAMAAAAzz3pWAAAANlBMVEX///8oKCgNDQ3MzMxUVFSI iIjc3NyYmJh2dna6urqqqqpmZmYyMjIiIiIQEBBERETu7u4AAAB0zXpNAAAAAXRSTlMAQObYZgAA AVVJREFUOMudlduagyAMhNmFcD6E93/ZRVAkLV2kXFT7GX6GGYmMbQ6pHhZaWS+YYnhOx7R63mmi 8Zl3z/FeLisuGvp2zfYxXa2V3DQNx6/J+LEWXsSGpfiBhrEu8o/1lqqV65RGmjenWVa5SxfNjvLg Wk067SDiB+svmi73WMwCabKaAi1ZLZ1FJpbpPs/enJGmSrg2IyBDNdfLIA0agz1NPapdX3moIDRb WC5Sf39/PBk5m/4stltX4wvQFfteQWi87C8ISDC6TR014aZf+JgqlPcp4p490goeizg5CHwxh/tx tWYOZl1tmFo/0kxgKrc89BxPT2mLFuvJEf5qK+OxI7QSrTg2WvbZ8aRlGZqLbn+P7aviiH5TT2lO s2Tr+wzzk+v4SxDNAgHApQDzVk9pnm+243VTGIMIu91ebTTXs6VtjfRcPvpt+vJz8p2SLz6G6qD/ AfFADZJOCE8IAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
2,Variable,,
3,Variable,,
4,Operation,operator: 5operands: 6,
5,Literal,,
6,ExprTuple,"7, 8","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACwAAAAUBAMAAAAEg8LyAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAQBJREFU GNNjYOC7wIAG2B4AidPIIqcjQKQcEK9DUbkSRJxiYOCdgCL8DUQwFzBwPEAWZfkDInkCGF6gKGZu AFNLGCqBJN/k6TEQYa4JbDeB2i0Z9gI5rg+4PkCEzyfU+QMdfIuhDWhULAPnBohw/vEDZ4FUBkMI AwPnHwaOAojwVIi79EHCrBsYzitAhBurCyDCQEPOGzDkQ0R5vvB9AdEeICvPJzBIgcOBgfcDzzdm BpCVlgwM3AW8jcwM5xcwMDBuYPgEEp4MVM/AI170sISBoweoIIGhsgYovAjJ8zlwv/I0IAWVASII gM7ZB1NSABc+hRQNvIgAA6nEjDTeBwwA1WI/NwU/k/MAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
7,Variable,,
8,Variable,,


This `lambda_map` may be accessed directly:

In [8]:
a_prod_range.lambda_map

k -> (a * k)

You can create the same **ExprRange** by proving `None` for the `parameter` and `body` but suppling the `lambda_map` instead:

In [9]:
from proveit import Lambda
a_prod_range_again = ExprRange(None, None, i, j, lambda_map=Lambda(k, Mult(a, k)))

a_prod_range_again: (a * i), ..(a * k).., (a * j)

In [10]:
a_prod_range_again == a_prod_range

True

When **ExprRange**(s) are used as *operands* of an **Operation**, it is typically formatted in a clean manner that reflects that there are an indeterminant number of *operands*:

In [11]:
add_with_ranges = Add(a_prod_range, b, a_prod_range)

add_with_ranges: (a * i) + ..(a * k).. + (a * j) + b + (a * i) + ..(a * k).. + (a * j)

Note that the *operands* are an **ExprTuple** that contains the **ExprRange**(s):

In [12]:
add_with_ranges.expr_info()

,core type,sub-expressions,expression
0,Operation,operator: 1operands: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAZcAAAAUBAMAAACzG238AAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAArFJREFU WMOtlzFrFEEUgJ/uGXJ7c2e6AyWSHyAkXdDKRrC0sBAxkEa0MocKcl2uUEgXSKEIQpQUZhGJpYWy zUGMB3u2FuFsJI0QjJwgUXwze7u3b2Z2992uW8ztzc43377ZmeENQKMP+nUX2JeFhqlBIa6kVeGf 9NqzcGLA7tag4ZYP5/RG1/O50laFv9HrZ0Fssrs1aPgMsKfXbedzpa0Sr69bGi9GE6aV0tvt0a+V Bqel4UMWt6g1mtCK+LTt43a5wVhpEFcpXjlmcV1uMHYr4ge2evcSM5gD++MtijurLC6y5gZjtyLe lpvDxpOb8ai28Q2cdWYwOg3i4Q6WFyjuNtvL2Ryx5gZjtyL+Hu8vD9zDeL5XcI7UN5nB6DRUYQnL LxQPXou/2Ryx5gZjtyL+GONagqocTyHrv1Wxy6lVZjAJGnxZ7KvFfofiKz78SuVMa24wOh1aEb+G cR3DtNQGcmT803h7UgbteN6rG543Y/R50fMeed4LeZugxW8VkVrs8xR/BuIoldOsiUYcq6JDK+L4 4NQOBHNyk5gZDWPYLefLJGjojBf7PMW3oTJM5Uxr7pcxaGVFHD9ZsAArY+Kt3EsnmGaUBleu4isU /yMHMptLWjnTzGJFHBdTsAxn4nxKDJ3JNgBCq7/GBvATuv1sjlg5G4DFijjuorVWveNE1ZXD7zgj F8jbqOWgikDz6XQ4X2CDBjML7yCbI9ZRo7GQZ0UcZ4Ro3vv6YJwk4SC6c/RtOlExmqJxtwYNT2Xx kuIf7/chh0tao0axkGdF3Joa7BZPZ9bkmKYuucyEZLd4OrMW4pkpn/BTuu2lpHznGz8SiaaJ9ziJ Zg8KWSX+wZIo8pNxSjeOatKzNyn3P6wKNw86UCt8TGo+T3nT/ONVWavCLUfQ/XIH2HqxY3NJK+L/ AFEtLlFHcG9gAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,ExprTuple,"4, 3, 4","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQwAAAAUBAMAAACKUB7iAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAntJREFU SMfFVr9rFEEU/s67LHuXvc11B0okf4CYK73OJmCXFBaCiVwjWIg5/EE4bC6NPyDFQYqEiBjERhCN laRQ1kIWk2LXTgRlbUQLyZGTEySK82b2djc7M9zBFpnizb15b7/35r3vZgawfRzxMAImdtOrJ0b9 ep5PuxeU1tzWiDCXHZxk0/P0+uSI35crYn6qTmPUIn8AdhhYJ2tN+5m7km/CDDJiFA4yp2HN4Xvm rbSzk/QJWkzaq2vzUWatptb7R/2WEynubTGXqq2GbHbXtCjJaLCWiMp1vGZyJih1o80V5rTlv4QX sdYZEyT0nln/ZHNtRptGMhqKWGDyE+6wjBZQpJwsWv9W3NR9/87HA5r5lsvBhCjCooPfktlubGiJ EEYTMF84w6/gPMvoACZ1wqMEnAltUx45uEs4fzgaZkUHNmD1JLMVpqYYg2gCxuEMn6Y0xrbgTTHF rIS704yLsPZpXhbqmcGxUejLZkP7/xlEC2E4w6epKV4Ni7HbS201ezC6CT08tv7S/iTzeEUHczga SnRunSOKeg0cNwbHh9XP69LoI7d5Lany8YtIIZm9Rk2XRjIaV4midZZ5s7wcxS50f3KS8NZFP7h4 hY+VGJyVnZNpEtsK86ztKxBIOxQt5MAqqwes6vWvN+MrxYe5EraOs4X38B7zfrt0uno2plpb+L2/ 4SvMuc9iMUbgeCvpaFgn8Vh9mF+Vr1OJuGYn4WeoeK1YlIHvU6Xa6qtN7muaMMecvamEn5JP+eHA p+x9cbXhjYKM8tHhpvS92nbSz1Wl4Q4FtnvjVIUd1bOHnZDyUvr1kGv5ST/l48IfDlx9SJIqcfSP wHKA/7sDxQaTdZVsAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,Variable,,
4,ExprRange,lambda_map: 5start_index: 6end_index: 7,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJMAAAAUBAMAAAB7fhJGAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAdVJREFU OMuVVDFLw0AUfjVpTGrSdisoQn+BZNNuLoKbDm5W6Q8QG0SR4tIu6iBScHBwsKuL1kkclDhIhQ7J qIvURXSQFoUO0sF3L02jySXFQC6579333b3v7h1A0gb/sw6RzwcHI0rTj05ArBWlpNb9iJR3KOf+ wCSotSipRGAiLU0UrcoZPR0l9cRFkSLzkrmPkjrkokh54+YwG66ULEMIpcSiR8f5gaklA0CohkuN sth7bsscII0dIMoNfuZaiY6LC+IiGhjh+xgeHnEVLjykGreJsosLWQGFNphN9KqgjFQOl5ILaIwN J5QC7V8rVSDKEoDSA9lggW8ml8LfkU64VBx9PDVhj/1bNdJbMImCUvE6WFkWqbCmaA6XWgb1k1aY JmgGiIIJWjoUvaGX+IrRCapfIP2e7MyhoO1WAcYl93ipXWGo7WoXYjWvULFLlBwGDa0iuLjYwWpV 9L4RZB81gx47KFfwmNa9EuwBUeZxRGbjZdMrQdzZBLNOPnDtq7i27JtO8nfbUxnvFCvMDqRwC+eB 2rUALpm8wpGrDiWinPUALnDKecRsZ/uUW86NRK6rRiDQIN//5tHWr11Kk7NJNFgLBmzO1Rcr2S6F cyE///9CRsoPFTB50q+h5OUAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
5,Lambda,parameter: 12body: 8,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAF4AAAAUCAMAAAAzz3pWAAAANlBMVEX///8oKCgNDQ3MzMxUVFSI iIjc3NyYmJh2dna6urqqqqpmZmYyMjIiIiIQEBBERETu7u4AAAB0zXpNAAAAAXRSTlMAQObYZgAA AVVJREFUOMudlduagyAMhNmFcD6E93/ZRVAkLV2kXFT7GX6GGYmMbQ6pHhZaWS+YYnhOx7R63mmi 8Zl3z/FeLisuGvp2zfYxXa2V3DQNx6/J+LEWXsSGpfiBhrEu8o/1lqqV65RGmjenWVa5SxfNjvLg Wk067SDiB+svmi73WMwCabKaAi1ZLZ1FJpbpPs/enJGmSrg2IyBDNdfLIA0agz1NPapdX3moIDRb WC5Sf39/PBk5m/4stltX4wvQFfteQWi87C8ISDC6TR014aZf+JgqlPcp4p490goeizg5CHwxh/tx tWYOZl1tmFo/0kxgKrc89BxPT2mLFuvJEf5qK+OxI7QSrTg2WvbZ8aRlGZqLbn+P7aviiH5TT2lO s2Tr+w

For convenience, there is a `var_range` method for creating an **ExprRange** over an indexed variable.

In [13]:
from proveit import var_range, IndexedVar
var_range(x, one, n)

x_{1}, ..x_{_a}.., x_{n}

Because of the `relabeling` freedom of **Lambda** expressions (alpha conversion), this is treated as `__eq__` to any similar **ExprRange** using any `parameter`.

In [14]:
var_range(x, one, n) == ExprRange(k, IndexedVar(x, k), one, n)

True

In [15]:
var_range(x, one, n) == ExprRange(a, IndexedVar(x, a), one, n)

True

Also note that there are a variety of variable ranges already defined for convenience 
in <a class="ProveItLink" href="../packages/proveit/core_expr_types/_theory_nbs_/common.ipynb">proveit.core\_expr\_types.\_theory\_nbs\_/common.ipynb</a>:

In [16]:
from proveit.core_expr_types import a_i_to_j, a_1_to_n, x_1_to_n

In [17]:
ExprTuple(a_i_to_j, a_1_to_n, x_1_to_n)

(a_{i}, ..a_{_a}.., a_{j}, a_{1}, ..a_{_a}.., a_{n}, x_{1}, ..x_{_a}.., x_{n})

### Showing the explicit parameterization

Some **ExprRanges** can be ambiguous in the default formatting style.  For example, the following expression are different but have identical default formatting

In [18]:
range__add1__0_to_n = ExprRange(k, Add(k, one), zero, n)

range__add1__0_to_n: (0 + 1), ..(k + 1).., (n + 1)

In [19]:
range__0_plus_1__to__n_plus_1 = ExprRange(k, k, Add(zero, one), Add(n, one))

range__0_plus_1__to__n_plus_1: (0 + 1), ..k.., (n + 1)

In [20]:
range__add1__0_to_n==range__0_plus_1__to__n_plus_1

False

We have an alternate *style* for formatting an **ExprRange** without this ambiguity.  This style is called *explicit parameterization*.  In between the first and last of the range, it shows the body of the lambda map of the **ExprRange**.

In [21]:
range__add1__0_to_n.with_explicit_parameterization()

(0 + 1), ..(k + 1).., (n + 1)

In [22]:
range__0_plus_1__to__n_plus_1.with_explicit_parameterization()

(0 + 1), ..k.., (n + 1)

Because it is less compact and not typically necessary, this is not the default style for LaTeX formatting.  However, it is the default for "string" formatting that gets used in error messages for the sake of clarity and making the debugging process a little easier.

### Simple repetitions

When an **ExprRange** has a lambda *body* that is independent of its parameter, it requires special formatting to avoid ambiguity.  The simple case in when the start index is $1$.

In [23]:
ExprRange(k, Add(one, two), one, n)

(1 + 2), ..n repeats.., (1 + 2)

Otherwise, the start and end indices will be provided when the **ExprRange** is formatted.

In [24]:
ExprRange(k, Add(one, two), i, j)

(1 + 2), ..from i to j.., (1 + 2)

### Nested ExprRanges

**ExprRange**s may be nested to express a range of ranges representing elements of a single **ExprTuple**.  For clarity of the expression, ellipses are doubled to express a range of ranges (or tripled for a range of ranges of ranges, etc.).

In [25]:
from proveit.numbers import one
double_nested = ExprRange(i, ExprRange(j, Mult(i, j), one, n), one, n)

double_nested: (1 * 1), ..(1 * j).., (1 * n), ....(i * 1), ..(i * j).., (i * n)...., (n * 1), ..(n * j).., (n * n)

In [26]:
ExprTuple(double_nested)

((1 * 1), ..(1 * j).., (1 * n), ....(i * 1), ..(i * j).., (i * n)...., (n * 1), ..(n * j).., (n * n))

If $n = 3$, for example, the above **ExprTuple** could expand to $((1 \cdot 1), (1 \cdot 2), (1 \cdot 3), (2 \cdot 1), (2 \cdot 2), (2 \cdot 3), (3 \cdot 1), (3 \cdot 2), (3 \cdot 3))$.

Let's now show triple nesting for fun:

In [27]:
triple_nested = ExprRange(i, ExprRange(j, ExprRange(k, Mult(i, j, k), 
                                                    one, n), 
                                       one, n), 
                          one, n)

triple_nested: (1 * 1 * 1), ..(1 * 1 * k).., (1 * 1 * n), ....(1 * j * 1), ..(1 * j * k).., (1 * j * n)...., (1 * n * 1), ..(1 * n * k).., (1 * n * n), ......(i * 1 * 1), ..(i * 1 * k).., (i * 1 * n), ....(i * j * 1), ..(i * j * k).., (i * j * n)...., (i * n * 1), ..(i * n * k).., (i * n * n)......, (n * 1 * 1), ..(n * 1 * k).., (n * 1 * n), ....(n * j * 1), ..(n * j * k).., (n * j * n)...., (n * n * 1), ..(n * n * k).., (n * n * n)

In [28]:
Add(triple_nested)

(1 * 1 * 1)+..(1 * 1 * k)..+(1 * 1 * n)+....(1 * j * 1)+..(1 * j * k)..+(1 * j * n)....+(1 * n * 1)+..(1 * n * k)..+(1 * n * n) + ......(i * 1 * 1)+..(i * 1 * k)..+(i * 1 * n)+....(i * j * 1)+..(i * j * k)..+(i * j * n)....+(i * n * 1)+..(i * n * k)..+(i * n * n)...... + (n * 1 * 1)+..(n * 1 * k)..+(n * 1 * n)+....(n * j * 1)+..(n * j * k)..+(n * j * n)....+(n * n * 1)+..(n * n * k)..+(n * n * n)

The `var_range` function may also be used to make nested ranges of indexed variables by simply supplying multiple start and end indices.

In [29]:
x_111_to_nnn = var_range(x, (one, one, one), (n, n, n))

x_111_to_nnn: x_{1, 1, 1}, ..x_{1, 1, _c}.., x_{1, 1, n}, ....x_{1, _b, 1}, ..x_{1, _b, _c}.., x_{1, _b, n}...., x_{1, n, 1}, ..x_{1, n, _c}.., x_{1, n, n}, ......x_{_a, 1, 1}, ..x_{_a, 1, _c}.., x_{_a, 1, n}, ....x_{_a, _b, 1}, ..x_{_a, _b, _c}.., x_{_a, _b, n}...., x_{_a, n, 1}, ..x_{_a, n, _c}.., x_{_a, n, n}......, x_{n, 1, 1}, ..x_{n, 1, _c}.., x_{n, 1, n}, ....x_{n, _b, 1}, ..x_{n, _b, _c}.., x_{n, _b, n}...., x_{n, n, 1}, ..x_{n, n, _c}.., x_{n, n, n}

Note that multiple indices are implemented by nesting **IndexedVar** expressions:

In [30]:
x_111_to_nnn.body.body.body.expr_info()

,core type,sub-expressions,expression
0,IndexedVar,variable: 1indices: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAPBAMAAABq0O6/AAAAMFBMVEX///8QEBBmZmaqqqrMzMxE REQyMjIiIiKYmJh2dna6urrc3NxUVFSIiIju7u4AAAA6P00QAAAAAXRSTlMAQObYZgAAAOhJREFU KM9jYHh7Z9fNCwzEAL4JHLXsCVgkuNLQRTgfcP7j2oDNkEkYpjKwBmC3LxlTiB+HS39jCvk7wJmP 7qTA2Sx/7zxAKJp7l4GB+cF9Bl6oGF8Jyz+4JG8drwKcw77AkYFhvUE8ww6oAJMCSwFclvUCByJg VBlYGBje3H2r0QAVYDTgPQCXZVzAhOAIo7v6/QbWPrj7+B8wGuD2IuMG/xUMW6GK2R+0MnDHQWWy GJjgoaukpKTMwHXWVoPB0QDCYTk5gYFFmOERkGPAsOnGPWzhaYDEZsabKJiRY9kar1IOZE4DEhsA vZY3Be67+CEAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,ExprTuple,"3, 4, 5","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGUAAAAUBAMAAABmLO6YAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAT1JREFU OMtjYOC7wEAKyAMRp3FK82xtwBRkfAAk1uE2k2sCFoMWMDDwTsCthz8Bi6A5AwPHA9x6/A9gETzK wPACj38tsDrYgaESweObPD0GRXph+TQ0d5UrMDBPYNiLEHF9wPUBxbu/DpxHcfmLBO4FDLwLGNoQ SmIZODcgK2H7w8BfgCzQzMCYwMDWwBACF+H8w8CBooRzAcN5A2T+HxDJ9AFJD+sGhvMKaNHjjxza rBugehBuA5qZjxo9FxjmI4c2xFKWBqQwOJ/AIMWG7Gd+B6AXUc1gSACFgSVciLuAt5GZ4fwCRNK6 wD0Bmc8pwHBSgYHTgMEDaNtuINjDwCNe9LCEgaMHITDdlgGZz2B8FxhGXAqYaSeHAP8EljRqQIAP TKMM+9DSegEB/gLMPMfLgJ/PDfILiXn7HgMDALX4VTDl8otIAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,Variable,,
4,Variable,,
5,Variable,,


## Expanding ExprRanges

The **ExprRange** and **IndexedVar** expression types work together in **ExprRange** expansion reduction rule.  Essentially, when a range of indexed variables is instantiated to an **ExprTuple** of elements and those indexed variables are contained within an **ExprRange**, that **ExprRange** must be "expanded" to reflect the expansion of the indexed variables.

### Parameter dependent expansions

The more restrictive form of **ExprRange** expansion is when **ExprRange** parameter is used for anything other than indices of indexed variables being expanded.  For example, consider the following lambda expression.

In [31]:
param_dep_lambda_example = Lambda(x_1_to_n, Add(ExprRange(k, Mult(k, IndexedVar(x, k)), 
                                                          one, n)))

param_dep_lambda_example: (x_{1}, ..x_{_a}.., x_{n}) -> ((1 * x_{1}) + ..(k * x_{k}).. + (n * x_{n}))

Note that the **ExprRange** parameter on the right appears as a prefactor as well as an index of $x$.  To respect the restrictions of a parameter dependent expansion, in this case, we may only apply this lambda map to an **ExprTuple** with **ExprRanges** whose indices match the original indices.  We demonstrate this below by applying `param_dep_lambda_example` to $a_1, \ldots, a_j, a_{j+1}, \ldots, a_n$.  First, we prove a few things in order to meet the necessary *requirements* for the expansion under a set of *assumption*

In [32]:
from proveit.core_expr_types import Len
from proveit.logic import InSet
from proveit.numbers import Natural, subtract
param_dep_assumptions = [InSet(j, Natural), InSet(subtract(n, j), Natural),
                         InSet(n, Natural)]
defaults.assumptions = param_dep_assumptions

defaults.assumptions: (j in Natural, (n - j) in Natural, n in Natural)

In [33]:
a_1_to_n_partition = a_1_to_n.partition(j)

a_1_to_n_partition: {j in Natural, (n - j) in Natural} |- (a_{1}, ..a_{_a}.., a_{n}) = (a_{1}, ..a_{_a}.., a_{j}, a_{j + 1}, ..a_{_a}.., a_{n})

In [34]:
typical = Len(a_1_to_n_partition.lhs).typical_eq()

typical: {n in Natural} |- |(a_{1}, ..a_{_a}.., a_{n})| = |(1, ..k.., n)|

In [35]:
from proveit.logic import Equals
equality = Equals(Len(a_1_to_n_partition.rhs), typical.rhs)

equality: |(a_{1}, ..a_{_a}.., a_{j}, a_{j + 1}, ..a_{_a}.., a_{n})| = |(1, ..k.., n)|

In [36]:
Len(a_1_to_n_partition.rhs).deduce_equality(equality)

{j in Natural, (n - j) in Natural, n in Natural} |- |(a_{1}, ..a_{x}.., a_{j}, a_{j + 1}, ..a_{x}.., a_{n})| = |(1, ..a.., n)|

In [37]:
typical.rhs.operand.entries[0].partition(j)

{j in Natural, (n - j) in Natural} |- (1, ..a.., n) = (1, ..a.., j, j + 1, ..a.., n)

In [38]:
param_dep_operands = a_1_to_n_partition.rhs

param_dep_operands: (a_{1}, ..a_{_a}.., a_{j}, a_{j + 1}, ..a_{_a}.., a_{n})

Now we will apply `param_dep_lambda_example` to $a_1, \ldots, a_j, a_{j+1}, \ldots, a_n$

In [39]:
requirements = []
param_dep_lambda_example.apply(*param_dep_operands.entries, requirements = requirements)

(1 * a_{1}) + ..(k * a_{k}).. + (j * a_{j}) + ((j + 1) * a_{j + 1}) + ..(k * a_{k}).. + (n * a_{n})

The following requirements are needed for this lambda application.  First, the length of the operands must match the length of the parameter index range for a proper applications (as we saw in the previous chapter).  Second, the indices themselves must match in order to perform this parameter-dependent **ExprRange** expansion.

In [40]:
requirements

### Parameter independent expansions

When the **ExprRange** parameter only appears as indices of variables that are being expanded, the **ExprRange** expansion reduction is more permissive.  Indices do not need to match.  Consider the following `dot_prod_lambda` example.

In [41]:
from proveit.core_expr_types import (
    a_1_to_j, b_1_to_k, c_1_to_j, d_1_to_k, x_1_to_n, y_1_to_n)
dot_prod_lambda = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

dot_prod_lambda: (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n}))

We map expand the $x_k$ variables and $y_k$ variables in any manner, with respecting the original indices, as long as the expansions of these two match.  We will demonstrate this by applying `dot_prod_lambda` such that the $x$ operands will be $(a_1, \dots, a_j, b_1, \ldots, b_k)$ and the $y$ operands will be $(c_1, \ldots, c_j, 1 \cdot d_1, \ldots, k \cdot d_k)$.  These don't respect the original indices since they go up to $j$ and then restart at $1$, but the do match each other, both going $1$ to $j$ then $1$ to $k$.  First, we will make some assumptions and prove a few things so we can meet the requirements.

In [42]:
jpk_eq_n = Equals(Add(j, k), n)

jpk_eq_n: (j + k) = n

In [43]:
param_indep_assumptions = [InSet(j, Natural), InSet(k, Natural), jpk_eq_n]
defaults.assumptions = param_indep_assumptions

defaults.assumptions: (j in Natural, k in Natural, (j + k) = n)

In [44]:
jpk_in_nats = InSet(Add(j, k), Natural).prove()

jpk_in_nats: {j in Natural, k in Natural} |- (j + k) in Natural

In [45]:
jpk_eq_n.sub_right_side_into(jpk_in_nats)

{j in Natural, k in Natural, (j + k) = n} |- n in Natural

In [46]:
x_operands = [a_1_to_j, b_1_to_k]

x_operands: (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k})

Make this one a little more interesting and demonstrate that expansions can be **ExprRange**s of any kind, not just variable ranges.

In [47]:
y_operands = [c_1_to_j, ExprRange(i, Mult(i, IndexedVar(d, i)), one, k)]

y_operands: (c_{1}, ..c_{_a}.., c_{j}, 1 * d_{1}, ..i * d_{i}.., k * d_{k})

In [48]:
x_len_jpk = Len(x_operands).computation()

x_len_jpk: {j in Natural, k in Natural} |- |(a_{1}, ..a_{x}.., a_{j}, b_{1}, ..b_{x}.., b_{k})| = (j + k)

In [49]:
jpk_eq_n.sub_right_side_into(x_len_jpk)

{j in Natural, k in Natural, (j + k) = n} |- |(a_{1}, ..a_{x}.., a_{j}, b_{1}, ..b_{x}.., b_{k})| = n

In [50]:
y_len_jpk = Len(y_operands).computation()

y_len_jpk: {j in Natural, k in Natural} |- |(c_{1}, ..c_{x}.., c_{j}, 1 * d_{1}, ..x * d_{x}.., k * d_{k})| = (j + k)

In [51]:
jpk_eq_n.sub_right_side_into(y_len_jpk)

{j in Natural, k in Natural, (j + k) = n} |- |(c_{1}, ..c_{x}.., c_{j}, 1 * d_{1}, ..x * d_{x}.., k * d_{k})| = n

Now we perform the lambda application

In [52]:
requirements = []
dot_prod_lambda.apply(*x_operands, *y_operands, requirements=requirements)

(x_{1} * y_{1}) + ..(x_{_a} * y_{_a}).. + (x_{j} * y_{j}) + (x_{1} * y_{1}) + ..(x_{_a} * y_{_a}).. + (x_{k} * y_{k})

Note that the requirements, in this case, only relate to matching the length of the operands and respective parameter index ranges.  The necessity that the $x$ operands and the $y$ operands match with respect to corresponding **ExprRange** start and end indices is not needed as an explicit requirement; it is a simple matter of checking that the expressions of these indices match identically.  In other words, that requirement is met by a trivial inspection.

In [53]:
requirements

Parameter independent expansions are only possible when all of the indexed variables of a particular expression range are expanded.

In [54]:
dot_prod_with_y = \
    Lambda(x_1_to_n, 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

dot_prod_with_y: (x_{1}, ..x_{_a}.., x_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n}))

In [55]:
from proveit import LambdaApplicationError
try:
    dot_prod_with_y.apply(a_1_to_j, b_1_to_k, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n})) to (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}) assuming (j in Natural, k in Natural, (j + k) = n):
Improper replacement: Improper replacement of (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{n} * y_{n}) via {(x_{1}, ..x_{_a}.., x_{n}): (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}), x: {(x_{1}, ..x_{_a}.., x_{n})}}:
ExprRange indices failed to match expansion which is necessary because not all of the indexed variables being indexed by the ExprRange parameter are being expanded (x is expanded but y is not): Unable to prove (1, .._a.., j, 1, .._a.., k) = (1, .._a.., n) assuming {j in Natural, k in Natural, (j + k) = n}:
'conclude' method not implemented for proof automation. 


But we can do a parameter dependent expansion in this case, as long as we can prove that the indices of the expansion match the original indices.

In [56]:
requirements = []
dot_prod_with_y.apply(*param_dep_operands.entries, assumptions=param_dep_assumptions, 
                      requirements=requirements)

(x_{1} * y_{1}) + ..(x_{_a} * y_{_a}).. + (x_{j} * y_{j}) + (x_{j + 1} * y_{j + 1}) + ..(x_{_a} * y_{_a}).. + (x_{n} * y_{n})

In [57]:
requirements

Also note that an explicit expansion must be provided for each precise range containing an expanded range of variables.

In [58]:
inconsistent_dot_prod = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, m)))

inconsistent_dot_prod: (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{m} * y_{m}))

In [59]:
try:
    inconsistent_dot_prod.apply(*x_operands, *y_operands, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

problem self (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{m} * y_{m}) {(x_{1}, ..x_{_a}.., x_{n}): (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}), (y_{1}, ..y_{_a}.., y_{n}): (c_{1}, ..c_{_a}.., c_{j}, 1 * d_{1}, ..i * d_{i}.., k * d_{k}), x: {(x_{1}, ..x_{_a}.., x_{n})}, y: {(y_{1}, ..y_{_a}.., y_{n})}}
Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{m} * y_{m})) to (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}, c_{1}, ..c_{_a}.., c_{j}, (1 * d_{1}), ..(i * d_{i}).., (k * d_{k})) assuming (j in Natural, k in Natural, (j + k) = n):
Improper replacement: Improper replacement of (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{m} * y_{m}) via {(x_{1}, ..x_{_a}.., x_{n}): (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}), (y_{1}, ..y_{_a}.., y_{n}): (c_{1}, ..c_{_a}.., c_{j}, 1 * d_{1}, ..i * d_{i}.., k * d_{k}), x: {(x_{1}, ..x_{_a}.., x_{n})}, y: {(y_{1}, ..y_{_a}.., y_{n})}}:
Failure to expand (x

However, we may supply *equivalent alternative expansions*, `equiv_alt_expansions`, to handle such cases as appropriate.  This was covered in the previous chapter and will not be revisited here.

All of the expansions within an **ExprRange** must match with respect to the indices of their **ExprRanges**.  That means that the number of entries of the expansions must match:

In [60]:
try:
    dot_prod_lambda.apply(a_1_to_j, b_1_to_k, y_1_to_n, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n})) to (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}, y_{1}, ..y_{_a}.., y_{n}) assuming (j in Natural, k in Natural, (j + k) = n):
Improper replacement: Improper replacement of (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{n} * y_{n}) via {(x_{1}, ..x_{_a}.., x_{n}): (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}), (y_{1}, ..y_{_a}.., y_{n}): (y_{1}, ..y_{_a}.., y_{n}), x: {(x_{1}, ..x_{_a}.., x_{n})}, y: {(y_{1}, ..y_{_a}.., y_{n})}}:
When expanding IndexedVar's within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}) vs (y_{1}, ..y_{_a}.., y_{n},) do not match as respected expansions for x_{a} and y_{a}. 


They must match with respect to which entries are singular versus **ExprRanges**:

In [61]:
from proveit.numbers import NaturalPos
defaults.assumptions = [InSet(n, NaturalPos)] + param_indep_assumptions

defaults.assumptions: (n in NaturalPos, j in Natural, k in Natural, (j + k) = n)

In [62]:
x_partition = x_1_to_n.partition(one)

x_partition: {n in NaturalPos} |- (x_{1}, ..x_{a}.., x_{n}) = (x_{1}, x_{1 + 1}, ..x_{a}.., x_{n})

In [63]:
x_operands_new = x_partition.rhs

x_operands_new: (x_{1}, x_{1 + 1}, ..x_{a}.., x_{n})

In [64]:
dot_prod_lambda

(x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n}))

In [65]:
try:
    dot_prod_lambda.apply(*x_operands_new.entries, a_1_to_j, b_1_to_k, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n})) to (x_{1}, x_{1 + 1}, ..x_{a}.., x_{n}, a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}) assuming (n in NaturalPos, k in Natural, j in Natural, (j + k) = n):
Improper replacement: Improper replacement of (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{n} * y_{n}) via {(x_{1}, ..x_{_a}.., x_{n}): (x_{1}, x_{1 + 1}, ..x_{a}.., x_{n}), (y_{1}, ..y_{_a}.., y_{n}): (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}), x: {(x_{1}, ..x_{_a}.., x_{n})}, y: {(y_{1}, ..y_{_a}.., y_{n})}}:
When expanding IndexedVar's within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (x_{1}, x_{1 + 1}, ..x_{a}.., x_{n}) vs (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}) do not match as respected expansions for x_{a} and y_{a}. 


Each of the start indices must match:

In [66]:
from proveit.numbers import zero
x_shift_equiv = x_1_to_n.shift_equivalence(new_start=zero)

x_shift_equiv: {n in NaturalPos} |- (x_{1}, ..x_{a}.., x_{n}) = (x_{0 + 1}, ..x_{i + 1}.., x_{(n - 1) + 1})

In [67]:
x_operands_shifted = x_shift_equiv.rhs

x_operands_shifted: (x_{0 + 1}, ..x_{i + 1}.., x_{(n - 1) + 1})

In [68]:
try:
    dot_prod_lambda.apply(*x_operands_shifted.entries, y_1_to_n, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n})) to (x_{0 + 1}, ..x_{i + 1}.., x_{(n - 1) + 1}, y_{1}, ..y_{_a}.., y_{n}) assuming (n in NaturalPos, k in Natural, j in Natural, (j + k) = n):
Improper replacement: Improper replacement of (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{n} * y_{n}) via {(x_{1}, ..x_{_a}.., x_{n}): (x_{0 + 1}, ..x_{i + 1}.., x_{(n - 1) + 1}), (y_{1}, ..y_{_a}.., y_{n}): (y_{1}, ..y_{_a}.., y_{n}), x: {(x_{1}, ..x_{_a}.., x_{n})}, y: {(y_{1}, ..y_{_a}.., y_{n})}}:
When expanding IndexedVar's within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (x_{0 + 1}, ..x_{i + 1}.., x_{(n - 1) + 1},) vs (y_{1}, ..y_{_a}.., y_{n},) do not match as respected expansions for x_{a} and y_{a}. 


And each of the end indices must match:

In [69]:
defaults.assumptions = param_indep_assumptions

defaults.assumptions: (j in Natural, k in Natural, (j + k) = n)

In [70]:
y_operands_new = [b_1_to_k, a_1_to_j]

y_operands_new: (b_{1}, ..b_{_a}.., b_{k}, a_{1}, ..a_{_a}.., a_{j})

In [71]:
ynew_len_kpj = Len(y_operands_new).computation()

ynew_len_kpj: {k in Natural, j in Natural} |- |(b_{1}, ..b_{x}.., b_{k}, a_{1}, ..a_{x}.., a_{j})| = (k + j)

In [72]:
ynew_len_jpk = ynew_len_kpj.inner_expr().rhs.commute()

ynew_len_jpk: {k in Natural, j in Natural} |- |(b_{1}, ..b_{x}.., b_{k}, a_{1}, ..a_{x}.., a_{j})| = (j + k)

In [73]:
jpk_eq_n.sub_right_side_into(ynew_len_jpk)

{k in Natural, j in Natural, (j + k) = n} |- |(b_{1}, ..b_{x}.., b_{k}, a_{1}, ..a_{x}.., a_{j})| = n

In [74]:
try:
    dot_prod_lambda.apply(*x_operands, *y_operands_new, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n})) to (a_{1}, ..a_{_a}.., a_{j}, b_{1}, ..b_{_a}.., b_{k}, b_{1}, ..b_{_a}.., b_{k}, a_{1}, ..a_{_a}.., a_{j}) assuming (j in Natural, k in Natural, (j + k) = n):
Failed to prove operand length requirement: Unable to prove (k + j) = n assuming {j in Natural, k in Natural, (j + k) = n}:
Unable to automatically conclude by standard means.  To try to prove this via transitive relations, try 'conclude_via_transitivity'.


## Nested ExprRange expansions

As we saw above, we may have nested **ExprRange** expressions and ranges of ranges of variables/parameters.

In [75]:
x_11_to_mn = var_range(x, (one, one), (m, n))

x_11_to_mn: x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n}

Consider the following two different lambda expressions mapping a range of ranges of parameters.  The first multiplies groups of sums of terms and the second is a range of ranges of terms added together. 

In [76]:
double_nested_lambda1 = Lambda(x_11_to_mn, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (i, j))), 
                               one, n)), 
              one, m)))

double_nested_lambda1: (x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n}) -> (((1 * 1 * x_{1, 1}) + ..(1 * j * x_{1, j}).. + (1 * n * x_{1, n})) * ..((i * 1 * x_{i, 1}) + ..(i * j * x_{i, j}).. + (i * n * x_{i, n})).. * ((m * 1 * x_{m, 1}) + ..(m * j * x_{m, j}).. + (m * n * x_{m, n})))

In [77]:
double_nested_lambda2 = Lambda(x_11_to_mn, Add(
    ExprRange(i, ExprRange(j, Mult(i, j, IndexedVar(x, (i, j))), 
                           one, n), 
              one, m)))

double_nested_lambda2: (x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n}) -> ((1 * 1 * x_{1, 1})+..(1 * j * x_{1, j})..+(1 * n * x_{1, n}) + ....(i * 1 * x_{i, 1})+..(i * j * x_{i, j})..+(i * n * x_{i, n}).... + (m * 1 * x_{m, 1})+..(m * j * x_{m, j})..+(m * n * x_{m, n}))

First, let's consider a simple example in which we assume that $m=2$ and $n=2$ and apply `double_nested_lambda` to $(a, b, c, d)$.

In [78]:
abcd = ExprTuple(a, b, c, d)

abcd: (a, b, c, d)

We'll make some assumptions and prove a few things to meet the requirements.

In [79]:
from proveit.numbers import two
defaults.assumptions = [Equals(n, two), Equals(m, two)]

defaults.assumptions: (n = 2, m = 2)

In [80]:
from proveit import extract_var_tuple_indices
Equals(Len(abcd), Len(extract_var_tuple_indices(ExprTuple(x_11_to_mn)))).prove()

{m = 2, n = 2} |- |(a, b, c, d)| = |((1, 1), ..(1, x).., (1, n), ....(k, 1), ..(k, x).., (k, n)...., (m, 1), ..(m, x).., (m, n))|

In [81]:
eq = Equals(ExprTuple(one, two), ExprTuple(ExprRange(k, k, one, two))).prove()

eq: |- (1, 2) = (1, ..k.., 2)

In [82]:
Equals(n, two).sub_left_side_into(eq.inner_expr().rhs[0].end_index)

{n = 2} |- (1, 2) = (1, ..k.., n)

In [83]:
Equals(m, two).sub_left_side_into(eq.inner_expr().rhs[0].end_index)

{m = 2} |- (1, 2) = (1, ..k.., m)

Now we will apply `double_nested_lambda1` to $(a, b, c, d)$ and demonstrate the expansion of nested **ExprRanges**.

In [84]:
requirements=[]
double_nested_lambda1.apply(*abcd.entries, requirements=requirements)

((1 * 1 * x_{1, 1}) + (1 * 2 * x_{1, 2})) * ((2 * 1 * x_{2, 1}) + (2 * 2 * x_{2, 2}))

Note in the requirements below that we need indices to match, (1,2)=(1,…,𝑛)
and (1,2)=(1,…,𝑚) in order to perform this parameter-dependent expansion.

In [85]:
# remove duplicate requirements using an OrderedDict
from collections import OrderedDict
requirements = list(OrderedDict.fromkeys(requirements))
requirements

Now we will apply `double_nested_lambda2` to $(a, b, c, d)$ and demonstrate the expansion of nested **ExprRanges**.

In [86]:
requirements=[]
double_nested_lambda2.apply(*abcd.entries, requirements=requirements)

(1 * 1 * x_{1, 1}) + (1 * 2 * x_{1, 2}) + (2 * 1 * x_{2, 1}) + (2 * 2 * x_{2, 2})

Note in the requirements below that we need indices to match, $(1, 2) = (1, \ldots, n)$ and $(1, 2) = (1, \ldots, m)$ in order to perform this parameter-dependent expansion.

In [87]:
# remove duplicate requirements using an OrderedDict
from collections import OrderedDict
requirements = list(OrderedDict.fromkeys(requirements))
requirements

Let's do one more example to test this capability

In [88]:
double_nested_operands = ExprTuple(var_range(x, (one, one), (subtract(m, one), n)),
                                   ExprRange(k, Mult(k, k), one, n))

double_nested_operands: (x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m - 1, 1}, ..x_{m - 1, _b}.., x_{m - 1, n}, 1 * 1, ..k * k.., n * n)

In [89]:
defaults.assumptions = [InSet(m, NaturalPos), InSet(n, Natural)]

defaults.assumptions: (m in NaturalPos, n in Natural)

In [90]:
ExprRange(k, k, one, m).partition(subtract(m, one))

{m in NaturalPos} |- (1, ..a.., m) = (1, ..a.., m - 1, m)

In [91]:
requirements=[]
double_nested_lambda1.apply(*double_nested_operands.entries, requirements=requirements)

((1 * 1 * x_{1, 1}) + ..(1 * j * x_{1, j}).. + (1 * n * x_{1, n})) * ..((i * 1 * x_{i, 1}) + ..(i * j * x_{i, j}).. + (i * n * x_{i, n})).. * (((m - 1) * 1 * x_{m - 1, 1}) + ..((m - 1) * j * x_{m - 1, j}).. + ((m - 1) * n * x_{m - 1, n})) * ((m * 1 * x_{m, 1}) + ..(m * j * x_{m, j}).. + (m * n * x_{m, n}))

In [92]:
requirements

In [93]:
requirements=[]
double_nested_lambda2.apply(*double_nested_operands.entries, requirements=requirements)

(1 * 1 * x_{1, 1})+..(1 * j * x_{1, j})..+(1 * n * x_{1, n}) + ....(i * 1 * x_{i, 1})+..(i * j * x_{i, j})..+(i * n * x_{i, n}).... + ((m - 1) * 1 * x_{m - 1, 1})+..((m - 1) * j * x_{m - 1, j})..+((m - 1) * n * x_{m - 1, n}) + (m * 1 * x_{m, 1}) + ..(m * j * x_{m, j}).. + (m * n * x_{m, n})

In [94]:
requirements

### Row-major vs column-major order

By our convention we the linearize a range of ranges of parameters according to row-major ordering.  This is what we used in the example above and is what `var_range` produced.  A column-major ordering is allowed, however, as long as it is employed consistently.  We repeat one of the above examples using column-major order.

In [95]:
x_11_to_mn__column_major = ExprRange(i, ExprRange(j, IndexedVar(x, (j, i)), one, n), one, m)

x_11_to_mn__column_major: x_{1, 1}, ..x_{j, 1}.., x_{n, 1}, ....x_{1, i}, ..x_{j, i}.., x_{n, i}...., x_{1, m}, ..x_{j, m}.., x_{n, m}

In [96]:
double_nested_lambda1_column_major = Lambda(x_11_to_mn__column_major, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (j, i))), 
                               one, n)), 
              one, m)))

double_nested_lambda1_column_major: (x_{1, 1}, ..x_{j, 1}.., x_{n, 1}, ....x_{1, i}, ..x_{j, i}.., x_{n, i}...., x_{1, m}, ..x_{j, m}.., x_{n, m}) -> (((1 * 1 * x_{1, 1}) + ..(1 * j * x_{j, 1}).. + (1 * n * x_{n, 1})) * ..((i * 1 * x_{1, i}) + ..(i * j * x_{j, i}).. + (i * n * x_{n, i})).. * ((m * 1 * x_{1, m}) + ..(m * j * x_{j, m}).. + (m * n * x_{n, m})))

In [97]:
defaults.assumptions = [Equals(n, two), Equals(m, two)]

defaults.assumptions: (n = 2, m = 2)

In [98]:
requirements = []
double_nested_lambda1_column_major.apply(*abcd.entries, requirements=requirements)

((1 * 1 * x_{1, 1}) + (1 * 2 * x_{2, 1})) * ((2 * 1 * x_{1, 2}) + (2 * 2 * x_{2, 2}))

If the ordering is not consistent, such as using row-major order for the parameters but column-major order in the bodoy of the lambda expression, an error will occur.

In [99]:
double_nested_lambda1_mixed_order = Lambda(x_11_to_mn, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (j, i))), 
                               one, n)), 
              one, m)))

double_nested_lambda1_mixed_order: (x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n}) -> (((1 * 1 * x_{1, 1}) + ..(1 * j * x_{j, 1}).. + (1 * n * x_{n, 1})) * ..((i * 1 * x_{1, i}) + ..(i * j * x_{j, i}).. + (i * n * x_{n, i})).. * ((m * 1 * x_{1, m}) + ..(m * j * x_{j, m}).. + (m * n * x_{n, m})))

In [100]:
try:
    requirements = []
    double_nested_lambda1_mixed_order.apply(*abcd.entries, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

problem self ((1 * 1 * x_{1, 1}) + ..(1 * j * x_{j, 1}).. + (1 * n * x_{n, 1})), ..((i * 1 * x_{1, i}) + ..(i * j * x_{j, i}).. + (i * n * x_{n, i})).., ((m * 1 * x_{1, m}) + ..(m * j * x_{j, m}).. + (m * n * x_{n, m})) {(x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n}): (a, b, c, d), x: {(x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n})}}
Expected error: Failure to apply (x_{1, 1}, ..x_{1, _b}.., x_{1, n}, ....x_{_a, 1}, ..x_{_a, _b}.., x_{_a, n}...., x_{m, 1}, ..x_{m, _b}.., x_{m, n}) -> (((1 * 1 * x_{1, 1}) + ..(1 * j * x_{j, 1}).. + (1 * n * x_{n, 1})) * ..((i * 1 * x_{1, i}) + ..(i * j * x_{j, i}).. + (i * n * x_{n, i})).. * ((m * 1 * x_{1, m}) + ..(m * j * x_{j, m}).. + (m * n * x_{n, m}))) to (a, b, c, d) assuming (n = 2, m = 2):
Improper replacement: Improper replacement of ((1 * 1 * x_{1, 1}) + ..(1 * j * x_{j, 1}).. + (1 * n * x_{n, 1})), ..((i

## Singular and empty range reductions

When the length of an **ExprRange** is known to be empty or singular, in the process of performing a lambda application, the **ExprRange** will be eliminated appropriately within the **ExprTuple** in which it is contained.  For example,

In [101]:
tuple_with_range_lambda = Lambda(n, Add(a, ExprRange(i, Mult(i, i), one, n), c))

tuple_with_range_lambda: n -> (a + (1 * 1) + ..(i * i).. + (n * n) + c)

Applying this lambda to set $n : 0$ will eliminate this **ExprRange**, leaving just the $a$ and $c$ terms in this sum.

In [102]:
requirements = []
tuple_with_range_lambda.apply(zero, requirements=requirements)

a + c

The *requirement* for this reduction is a statement that the **ExprTuple** of the **ExprRange**, with $n$ replaced by $0$, is equal to the **ExprTuple**.  Note that this is not effecting a direct substitution of these **ExprTuple**s since the original **ExprRange** is embedded in an **ExprTuple** with other entries.  So this reduction requires special treatment compared with the *equivalence reductions* that will be discussed in a later chapter.

In [103]:
requirements

This reduction works as long as it is known that the `end_index + 1 = start index`.

In [104]:
tuple_with_ij_range_lambda = Lambda(j, Add(a, ExprRange(b, Mult(b, b), i, j), c))

tuple_with_ij_range_lambda: j -> (a + (i * i) + ..(b * b).. + (j * j) + c)

In [105]:
Add(subtract(i, one), one).simplification([InSet(i, Natural)])

{i in Natural} |- ((i - 1) + 1) = i

In [106]:
requirements = []
tuple_with_ij_range_lambda.apply(subtract(i, one), assumptions=[InSet(i, Natural)],
                                 requirements=requirements)

a + c

In [107]:
requirements

To reduce the **ExprRange** to a singular element, the `start_index` and `end_index` must match exactly.

In [108]:
requirements = []
tuple_with_ij_range_lambda.apply(i, requirements=requirements)

a + (i * i) + c

In [109]:
requirements

There are no other special **ExprRange** reductions, even if the expansion is obvious.  For example,

In [110]:
from proveit.numbers import two
tuple_with_range_lambda.apply(two)

a + (1 * 1) + ..(i * i).. + (2 * 2) + c

Depending upon the circumstance, making the explicit expansion may not be desirable.  Such replacements should be made explicitly via the **theory system** rather than complicating the **core derivation system**.

In [111]:
%end exprrange_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>